In [1]:

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
from tqdm import tqdm


In [2]:
d = {'INTJ':'You are thoughtful, rational, quick-witted and independent. However, sometimes you are known to be overly critical, and have a combative side to yourself.', 'INTP':'You are unique, creative, inventive and imaginative. However, sometimes you are known to be a bit insensitive and impatient with others.', 'ENTJ':'You are determined, charismatic, confident and authoritative. However, sometimes you are known to  be intolerant of other people\'s weaknesses, and slightly arrogant.', 'ENTP':'You are audacious, bold, playful and rebellious. However, sometimes you can find it difficult to focus, and dislike talking about practical matters.', 'INFJ':'You are creative, insightful, passionate and have strong morals. However, sometimes you are a bit of a perfectionist, and find it reluctant to open up to other people.','INFP':'You are empathetic, generous, creative and passionate. However, sometimes your goals are a bit unrealistic and you tend to lack focus sometimes.', 'ENFJ':'You are passionate, reliable, charismatic and very receptive. However, sometimes you can be overly empathetic and condescending toward other people.', 'ENFP' : 'You are enthusiastic, festive, good-natured and excellent at communicating. However, you sometimes focus on being a people pleaser and disorganized.', 'ISTJ': 'You are very responsible, strong-willed, calm and enforce order. However, you are known to be stubborn and are somewhat judgemental sometimes', 'ISFJ': 'You are reliable, observant, enthusiastic and supportive. However, you are known to be overly humble and tend to take things personally', 'ESTJ' : 'You are dedicated, strong-willed, loyal and reliable. However, you find it difficult to relax, or share what you\'re feeling with other people.', 'ESFJ': 'You are very loyal, sensitive to other people\'s feelings, and have strong practical skills. However, you are sometimes worried about your social status and tend to be vulnerable to criticism.', 'ISTP': 'You are spotaneous, rational, optimistic and know how to prioritize things. However, you are known to be stubborn and get bored very easily.',  'ISFP' : 'You are charming, imaginative, passionate and sensitive to others. However, you are fiercely independent and get stressed out pretty easily.', 'ESTP': 'You are perceptive, direct, bold and rational. However, you tend to be defiant and may sometimes miss the bigger picture in favor of smaller victories.', 'ESFP' : 'You are observant, practical, have excellent people skills and are fond of showmanship. However, you are very sensitive and sometimes avoid conflict entirely.'}


In [3]:
mbti_df = pd.read_csv("mbti_1.csv")



mind_names = ["E" , "I"]
mind = []  # E or I

energy_names = ["N",  "S"]
energy = [] # N or S

nature_names = ["F", "T"]
nature = [] # F or T

tactics_names = ["J", "P"]

tactics = [] # J OR P

for t in mbti_df.type:
    mind.append(mind_names.index(t[0]))
    energy.append(energy_names.index(t[1]))
    nature.append(nature_names.index(t[2]))
    tactics.append(tactics_names.index(t[3]))

mbti_df['mind'] = mind
mbti_df['energy'] = energy
mbti_df['nature'] = nature
mbti_df['tactics'] = tactics


In [4]:
from googletrans import Translator
import re
import nltk
from nltk.stem import WordNetLemmatizer


from nltk.corpus import stopwords


translator = Translator()


def translate_text(text):
    result=""
    try:
        for sentence in text.split("."):
            result=result+translator.translate(sentence,dest='ko').text
    
        print(result)
        return result
    except Exception:
        return text

def replace_text(text):
    lemmatizer=WordNetLemmatizer()
    stop_words = set(stopwords.words('english')) # Load stop words
    pers_types = ['INFP' ,'INFJ', 'INTP', 'INTJ', 'ENTP', 'ENFP', 'ISTP' ,'ISFP' ,'ENTJ', 'ISTJ','ENFJ', 'ISFJ' ,'ESTP', 'ESFP' ,'ESFJ' ,'ESTJ']
    pers_types = [p.lower() for p in pers_types]  
    try:
        
        text=re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',text)
        
        text=re.sub('[^0-9a-z]',' ',text)
        text=text.lower()    
        text = " ".join([word for word in text.split() if word not in stop_words]) # Remove stop words
        #print(len(sentence))
        
        for p in pers_types:
            text = re.sub(p, '', text)
        #print(len(sentence))
        
        text = lemmatizer.lemmatize(text) # Lemmatize words
    except:
        ''''''    
    return text    








In [5]:
mbti_df['posts']=mbti_df['posts'].apply(replace_text)
# mbti_df['posts']=mbti_df['posts'].apply(translate_text)
# mbti_df.to_csv('mbti_1_kor.csv')

# mbti_df.to_csv('mbti_1_replace.csv')

In [6]:
mbti_df

,type,posts,mind,energy,nature,tactics
0,INFJ,moments sportscenter top ten plays pranks hat...,1,0,0,0
1,ENTP,finding lack posts alarming ex boring position...,0,0,1,1
2,INTP,ood one course say know blessing curse oes abs...,1,0,1,1
3,INTJ,ear enjoyed conversation day soteric gabbing n...,1,0,1,0
4,ENTJ,ou fired hat another silly misconception hat a...,0,0,1,0
...,...,...,...,...,...,...
8670,ISFP,always think cats doms reason websites become ...,1,1,0,1
8671,ENFP,thread already exists someplace else heck dele...,0,0,0,1
8672,INTP,many questions things would take purple pill i...,1,0,1,1
8673,INFP,conflicted right comes wanting children honest...,1,0,0,1


In [7]:
from sklearn.model_selection import train_test_split
max_len=50000
max_words=10000
label_mind = mbti_df.mind
label_energy = mbti_df.energy
label_nature = mbti_df.nature
label_tactics = mbti_df.tactics
feature = mbti_df.posts

tokenizer=tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(feature)

feature=tokenizer.texts_to_sequences(feature)

max_length=max([len(seq) for seq in feature])
vocabulary_size=len(tokenizer.word_index)+1
embedding_dim=100


feature=tf.keras.preprocessing.sequence.pad_sequences(feature,maxlen=max_length)

label_mind = np.array(label_mind)
label_energy = np.array(label_energy)
label_nature = np.array(label_nature)
label_tactics = np.array(label_tactics)



feature_train, feature_test, labelm_train, labelm_test = train_test_split (feature, label_mind, test_size =.3 , random_state= 42, stratify= label_mind)
feature_train, feature_test, labele_train, labele_test = train_test_split (feature, label_energy, test_size =.3 , random_state= 42, stratify= label_energy)
feature_train, feature_test, labeln_train, labeln_test = train_test_split (feature, label_nature, test_size =.3 , random_state= 42, stratify= label_nature)
feature_train, feature_test, labelt_train, labelt_test = train_test_split (feature, label_tactics, test_size =.3 , random_state= 42, stratify= label_nature)



In [8]:

def createModel():
  model = tf.keras.Sequential()
  # model.add(hub_layer)
  model.add(tf.keras.layers.Embedding(input_dim=vocabulary_size,output_dim=embedding_dim,input_length=max_length))
  # model.add(tf.keras.layers.Embedding(vocab_len+1,embedding_dim,input_length=max_length))
  model.add(tf.keras.layers.Dense(16, activation='relu'))
  model.add(tf.keras.layers.Dense(1,))

  # model.summary()
  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [9]:
def normalize(res, range, minima):
  normalized_vals = []
  for arr in res:
    normalized_vals.append((arr[0] + abs(minima))/range)
  return normalized_vals

In [10]:
feature
# feature_val = feature_train[3036:]
# partial_feature_train = feature_train[:3036]

array([[    0,     0,     0, ...,   181,   121,    32],
       [    0,     0,     0, ...,    44,    44,   117],
       [    0,     0,     0, ...,  1264, 58338, 58339],
       ...,
       [    0,     0,     0, ...,  2609,   590,  1538],
       [    0,     0,     0, ...,  1301,  1197,   115],
       [    0,     0,     0, ...,    91,    56,    50]])

In [11]:
tokenizer.word_index
import csv
with open('word_index.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['word', 'index'])
    for word, index in tokenizer.word_index.items():
        writer.writerow([word, index])



In [12]:

# m_val = labelm_train[3036:]
# partial_m_train = labelm_train[:3036]

# e_val = labele_train[3036:]
# partial_e_train = labele_train[:3036]

# n_val = labeln_train[3036:]
# partial_n_train = labeln_train[:3036]

# t_val = labelt_train[3036:]
# partial_t_train = labelt_train[:3036]

In [13]:
def float_to_mind(float_results):
    res = []
    for num in float_results:
        if(num < 0.5):
            res.append("E")
        else:
            res.append("I")
    return res


def float_to_energy(float_results):
    res = []
    for num in float_results:
        if(num < 0.5):
            res.append("N")
        else:
            res.append("S")
    return res


def float_to_nature(float_results):
    res = []
    for num in float_results:
        if(num < 0.5):
            res.append("F")
        else:
            res.append("T")
    return res   

def float_to_tactics(float_results):
    res = []
    for num in float_results:
        if(num < 0.5):
            res.append("J")
        else:
            res.append("P")
    return res

In [14]:

mind = createModel()
history = mind.fit(feature,
                    label_mind,
                    epochs=40,
                    batch_size=512,
                     validation_split=0.2,
                    )

# tfjs.converters.save_keras_model(mind, "mind")

Epoch 1/40


ResourceExhaustedError: Graph execution error:

Detected at node 'Cast_19' defined at (most recent call last):
    File "c:\Users\taegi\anaconda3\envs\env38\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\taegi\anaconda3\envs\env38\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\taegi\anaconda3\envs\env38\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\taegi\anaconda3\envs\env38\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
      app.start()
    File "c:\Users\taegi\anaconda3\envs\env38\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\taegi\anaconda3\envs\env38\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\taegi\anaconda3\envs\env38\lib\asyncio\bas

In [ ]:
energy = createModel()
history = energy.fit(feature,
                    label_energy,
                    epochs=40,
                    batch_size=512,
                    validation_split=0.2,
                   )
# tfjs.converters.save_keras_model(energy, "energy")

Epoch 1/40
14/14 [==============================] - 32s 2s/step - loss: 1.1004 - accuracy: 0.8630 - val_loss: 0.7529 - val_accuracy: 0.8582
Epoch 2/40
14/14 [==============================] - 30s 2s/step - loss: 0.5457 - accuracy: 0.8630 - val_loss: 0.4603 - val_accuracy: 0.8582
Epoch 3/40
14/14 [==============================] - 35s 2s/step - loss: 0.4317 - accuracy: 0.8630 - val_loss: 0.4292 - val_accuracy: 0.8582
Epoch 4/40
14/14 [==============================] - 31s 2s/step - loss: 0.4125 - accuracy: 0.8630 - val_loss: 0.4161 - val_accuracy: 0.8582
Epoch 5/40
14/14 [==============================] - 31s 2s/step - loss: 0.4055 - accuracy: 0.8630 - val_loss: 0.4135 - val_accuracy: 0.8582
Epoch 6/40
14/14 [==============================] - 31s 2s/step - loss: 0.4037 - accuracy: 0.8630 - val_loss: 0.4123 - val_accuracy: 0.8582
Epoch 7/40
14/14 [==============================] - 32s 2s/step - loss: 0.4027 - accuracy: 0.8630 - val_loss: 0.4115 - val_accuracy: 0.8582
Epoch 8/40
14/14 [==

In [ ]:
nature = createModel()
history = nature.fit(feature,
                    label_nature,
                    epochs=40,
                    batch_size=512,
                    validation_split=0.2,
                   )
# tfjs.converters.save_keras_model(model, "nature")

Epoch 1/40
14/14 [==============================] - 31s 2s/step - loss: 1.2320 - accuracy: 0.5427 - val_loss: 0.9398 - val_accuracy: 0.5349
Epoch 2/40
14/14 [==============================] - 30s 2s/step - loss: 0.8516 - accuracy: 0.5427 - val_loss: 0.7948 - val_accuracy: 0.5349
Epoch 3/40
14/14 [==============================] - 30s 2s/step - loss: 0.7604 - accuracy: 0.5426 - val_loss: 0.7454 - val_accuracy: 0.5343
Epoch 4/40
14/14 [==============================] - 30s 2s/step - loss: 0.7290 - accuracy: 0.5419 - val_loss: 0.7259 - val_accuracy: 0.5346
Epoch 5/40
14/14 [==============================] - 30s 2s/step - loss: 0.7161 - accuracy: 0.5425 - val_loss: 0.7167 - val_accuracy: 0.5349
Epoch 6/40
14/14 [==============================] - 30s 2s/step - loss: 0.7091 - accuracy: 0.5427 - val_loss: 0.7111 - val_accuracy: 0.5349
Epoch 7/40
14/14 [==============================] - 30s 2s/step - loss: 0.7045 - accuracy: 0.5427 - val_loss: 0.7075 - val_accuracy: 0.5349
Epoch 8/40
14/14 [==

In [ ]:
tactics = createModel()
history = tactics.fit(feature,
                    label_tactics,
                    epochs=40,
                    batch_size=512,
                    validation_split=0.2,
                   )
#tfjs.converters.save_keras_model(model, "tactics")


Epoch 1/40
14/14 [==============================] - 33s 2s/step - loss: 8.1212 - accuracy: 0.3955 - val_loss: 7.0940 - val_accuracy: 0.3971
Epoch 2/40
14/14 [==============================] - 32s 2s/step - loss: 6.1324 - accuracy: 0.3955 - val_loss: 5.0575 - val_accuracy: 0.3971
Epoch 3/40
14/14 [==============================] - 32s 2s/step - loss: 2.9746 - accuracy: 0.3955 - val_loss: 1.7432 - val_accuracy: 0.3971
Epoch 4/40
14/14 [==============================] - 31s 2s/step - loss: 1.5738 - accuracy: 0.3955 - val_loss: 1.4371 - val_accuracy: 0.3971
Epoch 5/40
14/14 [==============================] - 31s 2s/step - loss: 1.3781 - accuracy: 0.3955 - val_loss: 1.3205 - val_accuracy: 0.3971
Epoch 6/40
14/14 [==============================] - 31s 2s/step - loss: 1.2830 - accuracy: 0.3955 - val_loss: 1.2443 - val_accuracy: 0.3971
Epoch 7/40
14/14 [==============================] - 31s 2s/step - loss: 1.2146 - accuracy: 0.3955 - val_loss: 1.1845 - val_accuracy: 0.3971
Epoch 8/40
14/14 [==

In [ ]:

# Mind
res1 = mind.predict(feature_train)
m_minima = float((min(res1))[0])
m_maxima = float((max(res1))[0])
m_range = m_maxima-m_minima

# Energy
res1 = energy.predict(feature_train)
e_minima = float((min(res1))[0])
e_maxima = float((max(res1))[0])
e_range = e_maxima-e_minima

# Nature
res1 = nature.predict(feature_train)
n_minima = float((min(res1))[0])
n_maxima = float((max(res1))[0])
n_range = n_maxima-n_minima

# Tactics
res1 = tactics.predict(feature_train)
t_minima = float((min(res1))[0])
t_maxima = float((max(res1))[0])
t_range = t_maxima-t_minima

190/190 [==============================] - 1s 5ms/step


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:

def predict(input_string):
  input_arr = []
  input_arr.append(input_string)

  # Mind
  results = mind.predict(input_arr)
  mind_res = (float_to_mind(normalize(results, m_range, m_minima)))[0]

  # Energy
  results = energy.predict(input_arr)
  energy_res = (float_to_energy(normalize(results, e_range, e_minima)))[0]
  
  # Nature
  results = nature.predict(input_arr)
  nature_res = (float_to_nature(normalize(results, n_range, n_minima)))[0]

  # Tactics
  results = tactics.predict(input_arr)
  tactics_res = (float_to_tactics(normalize(results, t_range, t_minima)))[0]

  return mind_res + energy_res + nature_res + tactics_res

In [ ]:
results = predict(input())
print(results)
print(d[results])

# mind.save('mind.h5')
# energy.save('energy.h5')
# nature.save('nature.h5')
# tactics.save('tactics.h5')

In [ ]:
# energy.save('./energys', save_format='tf')

# mind.save('./minds', save_format='tf')
# nature.save('./natures', save_format='tf')
# tactics.save('./tacticss', save_format='tf')


INFO:tensorflow:Assets written to: ./energys\assets


INFO:tensorflow:Assets written to: ./energys\assets


INFO:tensorflow:Assets written to: ./minds\assets


INFO:tensorflow:Assets written to: ./minds\assets


INFO:tensorflow:Assets written to: ./natures\assets


INFO:tensorflow:Assets written to: ./natures\assets


INFO:tensorflow:Assets written to: ./tacticss\assets


INFO:tensorflow:Assets written to: ./tacticss\assets
